# Trabalho Computacional 3. Rede Convolucional e Transfer Learning

## 1. Introdução e Base de Dados

Neste trabalho usaremos uma rede convolucional pré-treinada e a aplicaremos em um problema novo. Também experimentaremos com a
divisão da base em treinamento, validação e teste, e usaremos o conjunto de validação para a técnica "early stopping", na tentativa de controlar o sobre-ajuste.

A base de dados é a [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html). Ela contém 60000 imagens 32x32 coloridas (3 canais) das seguintes categorias de objetos: ‘airplane’, ‘automobile’, ‘bird’, ‘cat’, ‘deer’, ‘dog’, ‘frog’, ‘horse’, ‘ship’, ‘truck’.

Ela pode ser baixada com o código abaixo.

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

In [3]:
class CIFAR10():  #@save    
    def __init__(self, root, resize=(224, 224)):    
        trans = transforms.Compose([transforms.Resize(resize),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        self.train = torchvision.datasets.CIFAR10(
            root=root, train=True, transform=trans, download=True)
        # use 20% of training data for validation
        train_set_size = int(len(self.train) * 0.8)
        valid_set_size = len(self.train) - train_set_size
         # split the train set into two
        seed = torch.Generator().manual_seed(42)
        self.train, self.val = torch.utils.data.random_split(self.train, [train_set_size, valid_set_size], generator=seed)
        self.test = torchvision.datasets.CIFAR10(
            root=root, train=False, transform=trans, download=True)
        
dataset = CIFAR10(root="./data/")

train_dataloader = torch.utils.data.DataLoader(dataset.train, batch_size=64, shuffle=True, num_workers=5)
val_dataloader = torch.utils.data.DataLoader(dataset.val, batch_size=64, shuffle=False, num_workers=3)
test_dataloader = torch.utils.data.DataLoader(dataset.test, batch_size=64, shuffle=False, num_workers=3)

print(f"Number of training examples: {len(dataset.train)}")
print(f"Number of validation examples: {len(dataset.val)}")
print(f"Number of test examples: {len(dataset.test)}")

Number of training examples: 40000
Number of validation examples: 10000
Number of test examples: 10000


Observe como foi feita a separação de 10.000 exemplos do conjunto de treinamento original para serem o conjunto de validação. Dessa forma, temos ao final 40.000 exemplos para treinamento, 10.000 exemplos para validação e 10.000 exemplos para teste, com os seus respectivos `DataLoader`'s instanciados.

Também redimensionamos as imagens para 224x224pixels, já preparando o dado para a posterior aplicação na rede convolucional.

## 2. Treinando um MLP

Use esta base de dados para treinar um Perceptron Multicamadas, como feito no trabalho anterior com a base MNIST. Escolha um MLP com 2 camadas escondidas. Não perca muito tempo variando a arquitetura porque este problema é difícil sem o uso de convoluções e o resultado não será totalmente satisfatório.

Você pode usar este código, baseado na biblioteca [Pytorch Lightning](https://lightning.ai/docs/pytorch/stable/) como base para definição da rede:

In [4]:
%pip install pytorch-lightning
import pytorch_lightning as pl
import torch.nn as nn
from torchmetrics.functional import accuracy

# The model is passed as an argument to the `LightModel` class.
class LightModel(pl.LightningModule):
	def __init__(self,model,lr=1e-5):
		super().__init__()
		self.model = model
		self.lr = lr
	def training_step(self, batch):
		X, y = batch
		y_hat = self.model(X)
		loss = nn.functional.cross_entropy(y_hat, y)
		self.log("train_loss", loss)
		return loss
	def validation_step(self, batch):
		X, y = batch
		y_hat = self.model(X)
		loss = nn.functional.cross_entropy(y_hat, y)
		self.log("val_loss", loss)
		return loss
	def test_step(self, batch):
		X, y = batch
		y_hat = self.model(X)
		preds = torch.argmax(y_hat, dim=1)
		acc = accuracy(preds, y, task="multiclass", num_classes=10)
		self.log("test_acc", acc)
		loss = nn.functional.cross_entropy(y_hat, y)		
		self.log("test_loss", loss)		
	def configure_optimizers(self):
		optimizer = torch.optim.Adam(self.parameters(), self.lr)
		return optimizer


arch = nn.Sequential(
			nn.Flatten(),
			nn.Linear(3*224*224, 512),
			nn.ReLU(),
			nn.Linear(512,128),
			nn.ReLU(),
			nn.Linear(128,10)	
	)

mlp = LightModel(arch)


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
from pytorch_lightning.callbacks import ModelCheckpoint
import os
import glob

class LastNCheckpoints(ModelCheckpoint):
    def __init__(self, keep_last_n=5, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.keep_last_n = keep_last_n

    def on_validation_end(self, trainer, pl_module):
        super().on_validation_end(trainer, pl_module)

        # Clean up old checkpoints
        all_ckpts = sorted(
            glob.glob(os.path.join(self.dirpath, "*.ckpt")),
            key=os.path.getmtime
        )
        if len(all_ckpts) > self.keep_last_n:
            for ckpt in all_ckpts[:-self.keep_last_n]:
                os.remove(ckpt)


Observe que as imagens são achatadas (transformadas em vetor). Substitua as interrogações pelo tamanho desejado das camadas
escondidas.

Neste problema vamos verificar o fenômeno do sobreajuste, e vamos tentar equilibrá-lo pela técnica de parada prematura de treinamento (early-stopping).
Por isso foi necessário, a partir dos dados de treinamento, fazer uma nova separação para validação. Quando a função custo (`loss`) no conjunto de validação não diminui num dado número de épocas (o parâmetro `patience`), o treinamento é interrompido. Este trecho de código pode ser útil:

In [20]:
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning import Trainer
from pytorch_lightning.utilities.model_summary import summarize

checkpoint = LastNCheckpoints(
    dirpath='model1_checkpoints/',
    filename='{epoch:02d}-{val_loss:.4f}',
    every_n_epochs=1,  # Save every epoch
    save_top_k=-1  # Save ALL checkpoints
)

early_stopping = EarlyStopping(
    monitor='val_loss',  # metric to monitor
    patience=5,          # epochs with no improvement after which training will stop
    mode='min',          # mode for min loss; 'max' if maximizing metric
    min_delta=0.001      # minimum change to qualify as an improvement
)

trainer = Trainer(
    callbacks=[early_stopping, checkpoint],
    max_epochs=50,
)
mlp.train()
TRAIN = False
# TRAIN = True
if TRAIN:
    trainer.fit(
        model=mlp,
        train_dataloaders=train_dataloader,
        val_dataloaders=val_dataloader,
        ckpt_path="model1_checkpoints/checkpoint-epoch=15-val_loss=1.3520.ckpt"
    )
else:
    print("LOADING MODEL")

    checkpoint = torch.load("model1_checkpoints/epoch=16-val_loss=1.3592.ckpt", map_location=torch.device('cpu'))  # or 'cuda'

    # Load state dict into your model
    mlp.load_state_dict(checkpoint['state_dict'])
    
    summary = summarize(mlp, max_depth=2)  # max_depth controls how deep to show layers
    print(summary)

mlp.eval()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOADING MODEL
  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | model   | Sequential | 77.1 M | train
1 | model.0 | Flatten    | 0      | train
2 | model.1 | Linear     | 77.1 M | train
3 | model.2 | ReLU       | 0      | train
4 | model.3 | Linear     | 65.7 K | train
5 | model.4 | ReLU       | 0      | train
6 | model.5 | Linear     | 1.3 K  | train
-----------------------------------------------
77.1 M    Trainable params
0         Non-trainable params
77.1 M    Total params
308.551   Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


LightModel(
  (model): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=150528, out_features=512, bias=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=128, bias=True)
    (4): ReLU()
    (5): Linear(in_features=128, out_features=10, bias=True)
  )
)

Os parâmetros dados são sugestões. Você agora pode testar o seu modelo, por exemplo, com:

In [22]:
# Evaluate the model on the test dataset
trainer.test(model=mlp, dataloaders=test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|████████████████████████████████████████████████████████████████████████████████| 157/157 [00:06<00:00, 26.16it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.5370000004768372
        test_loss            1.365114688873291
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_acc': 0.5370000004768372, 'test_loss': 1.365114688873291}]

Mais uma vez, procure realizar ajustes, mas não espere um bom desempenho. Como dissemos, é um problema complexo de classificação de
imagem, e é difícil fazer o MLP funcionar sozinho. Precisamos de um pré-processamento com base em uma rede convolucional.

## 3. Uso da rede VGG16 pré-treinada

Lembre-se que a rede VGG usa como bloco básico cascata de convoluções com filtros 3x3, com "padding" para que a imagem não seja
diminuída, seguida de um "max pooling" reduzindo imagens pela metade. O número de mapas vai aumentando e seu tamanho vai diminuindo
ao longo de suas 16 camadas. Este é um modelo gigantesco e o treinamento com recursos computacionais modestos levaria dias ou
semanas, se é que fosse possível.

No entanto, vamos aproveitar uma característica central das grandes redes convolucionais. Elas podem ser usadas como pré-processamento
fixo das imagens, mesmo em um novo problema (lembre-se, a rede VGG original foi treinada na base ImageNet, que tem muitas categorias de
imagens).

O código abaixo realiza o download do modelo treinado e configura os seus parâmetros como não ajustáveis.

In [14]:
from torchvision.models import vgg16
vgg16_model = vgg16(weights="DEFAULT", progress=True)

for param in vgg16_model.parameters():
	param.requires_grad = False

print(vgg16_model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

Agora, modifique o bloco de classificação (`vgg16_model.classifier`) da VGG16 de forma que fique semelhante ao MLP anterior, para trabalhar com somente 10 classes. Especifique uma camada `Flatten` (o MLP espera um vetor de entradas), e duas camadas densas. Elas não precisam ser muito grandes. Experimente com 50 e 20 neurônios, respectivamente, ou algo próximo.

Volte a treinar e testar o modelo (pode usar as classes `LightModel` e `Trainer`). Mesmo sem efetivamente treinar toda a rede VGG (somente os parâmetros do bloco de classificação), ainda temos que passar os dados por ela a cada passo, e o treinamento é um tanto lento. Mas desta vez o problema deve ser resolvido satisfatoriamente.



In [15]:
import torch.nn as nn

print(vgg16_model.classifier)
vgg16_model.classifier = nn.Sequential(
    # nn.Flatten(),
    nn.Linear(25088, 50),
    nn.ReLU(),
    nn.Linear(50, 20),
    nn.ReLU(),
    nn.Linear(20, 10)
)

# make sure we unfreeze here for training:
for param in vgg16_model.classifier.parameters():
    param.requires_grad = True

print(vgg16_model.classifier)

cnn = LightModel(vgg16_model)

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)
Sequential(
  (0): Linear(in_features=25088, out_features=50, bias=True)
  (1): ReLU()
  (2): Linear(in_features=50, out_features=20, bias=True)
  (3): ReLU()
  (4): Linear(in_features=20, out_features=10, bias=True)
)


In [21]:
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning import Trainer
from pytorch_lightning.utilities.model_summary import summarize

checkpoint = LastNCheckpoints(
    dirpath='model2_checkpoints/',
    filename='{epoch:02d}-{val_loss:.4f}',
    every_n_epochs=1,  # Save every epoch
    save_top_k=-1  # Save ALL checkpoints
)

early_stopping = EarlyStopping(
    monitor='val_loss',  # metric to monitor
    patience=5,          # epochs with no improvement after which training will stop
    mode='min',          # mode for min loss; 'max' if maximizing metric
    min_delta=0.001      # minimum change to qualify as an improvement
)

trainer = Trainer(
    callbacks=[early_stopping, checkpoint],
    max_epochs=50,
)

cnn.train()

TRAIN = False
# TRAIN = True
if TRAIN:
    trainer.fit(
        model=cnn,
        train_dataloaders=train_dataloader,
        val_dataloaders=val_dataloader,
        # ckpt_path="model2_checkpoints/checkpoint-epoch=15-val_loss=1.3520.ckpt"
    )
else:
    print("LOADING MODEL")

    checkpoint = torch.load(
        "model2_checkpoints/epoch=34-val_loss=0.4055.ckpt",
        map_location=torch.device('cpu')
    )  # or 'cuda'
    
    # Load state dict into your model
    cnn.load_state_dict(checkpoint['state_dict'])

    summary = summarize(cnn, max_depth=2)  # max_depth controls how deep to show layers
    print(summary)

cnn.eval()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOADING MODEL
  | Name             | Type              | Params | Mode 
---------------------------------------------------------------
0 | model            | VGG               | 16.0 M | train
1 | model.features   | Sequential        | 14.7 M | train
2 | model.avgpool    | AdaptiveAvgPool2d | 0      | train
3 | model.classifier | Sequential        | 1.3 M  | train
---------------------------------------------------------------
1.3 M     Trainable params
14.7 M    Non-trainable params
16.0 M    Total params
63.881    Total estimated model params size (MB)
40        Modules in train mode
0         Modules in eval mode


LightModel(
  (model): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16

# Evaluate the model on the test dataset

In [22]:
trainer.test(model=cnn, dataloaders=test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|████████████████████████████████████████████████████████████████████████████████| 157/157 [01:37<00:00,  1.61it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc             0.859499990940094
        test_loss           0.42019277811050415
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_acc': 0.859499990940094, 'test_loss': 0.42019277811050415}]

## 4. Extras (opcionais)

### 4.1. Procure usar outra(s) redes convolucionais como base.
### 4.2. No lugar de "early stopping", experimente com regularização L1 e L2, e "dropout".